In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE

In [2]:
# Load cleaned data
data = pd.read_csv('clean_train.csv', sep=';', decimal=',')

In [3]:
# Split data to separate 'y' and 'X'.
y = data['EXTRA_BAGGAGE']

# Drop target variable from X DataFrame
X = data.drop('EXTRA_BAGGAGE', 1)

In [ ]:
# We'll apply nested cross-validation in order to avoid the possible bias of applying 
# cross-validation combined with grid-search to perform parameter tuning.

# We'll use nested cross-validation over several classification methods in order to
# obtain their generalization score and choose the one with highest score. After this,
# we'll perform a simple cross-validation with grid-search in order to obtain the 
# best configuration of hyper-parameters for this algorithm. Finally we'll train 
# the model with all the training data and the hyper-parameter configuration that
# yield the best results.

# variace: it removes useless variables with variace equal to zero.
# scaler: standardizes each column to have mean equal to zero and variance equal to 1, 
# in order to give same importance to variables with different measurement units.
lr_pipe = Pipeline([('variance', VarianceThreshold()),
                    ('scaler', StandardScaler()),
                    ('lr', LogisticRegression(class_weight='balanced', random_state=621473))])

param_grid = dict()
param_grid['lr__C'] = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

inner_cv = StratifiedKFold(n_splits=10, random_state=975428)
outer_cv = StratifiedKFold(n_splits=10, random_state=248733)

gs_cv = GridSearchCV(lr_pipe, param_grid=param_grid, n_jobs=-1, scoring='f1_weighted', cv=inner_cv, verbose=10)

lr_nested_cv_f1_scores = cross_val_score(gs_cv, X, y, cv=outer_cv, verbose=10)

joblib.dump(lr_nested_cv_f1_scores, 'lr_nested_cv_f1_scores.pkl', compress=1)

print("##### Generalization F1 Score: mean =", np.mean(lr_nested_cv_f1_scores), "std =", np.std(lr_nested_cv_f1_scores))
print()

##### Generalization F1 Score: mean = 0.663232566525 std = 0.00577941758843

In [ ]:
linear_svm_pipe = Pipeline([('variance', VarianceThreshold()),
                            ('scaler', StandardScaler()),
                            ('linear_svm', SVC(kernel='linear', random_state=123456, probability=True, class_weight='balanced'))])

param_grid = dict()
param_grid['linear_svm__C'] = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

inner_cv = StratifiedKFold(n_splits=10, random_state=975428)
outer_cv = StratifiedKFold(n_splits=10, random_state=248733)

gs_cv = GridSearchCV(linear_svm_pipe, param_grid=param_grid, n_jobs=-1, scoring='f1_weighted', cv=inner_cv, verbose=10)

linear_svm_nested_cv_f1_scores = cross_val_score(gs_cv, X, y, cv=outer_cv, verbose=10)

joblib.dump(linear_svm_nested_cv_f1_scores, 'linear_svm_nested_cv_f1_scores.pkl', compress=1)

print("##### Generalization F1 Score: mean =", np.mean(linear_svm_nested_cv_f1_scores), "std =", np.std(linear_svm_nested_cv_f1_scores))
print()

In [ ]:
rf_pipe = Pipeline([('variance', VarianceThreshold()),
                            ('scaler', StandardScaler()),
                            ('rf', RandomForestClassifier(max_features='sqrt', oob_score=True, random_state=573146, class_weight='balanced'))])

param_grid = dict()
param_grid['rf__n_estimators'] = list(range(200, 1200, 150))
param_grid['rf__max_depth'] = [10, 20, 30]
param_grid['rf__min_samples_leaf'] = [50, 60, 70, 80, 90, 100]


inner_cv = StratifiedKFold(n_splits=10, random_state=975428)
outer_cv = StratifiedKFold(n_splits=10, random_state=248733)

gs_cv = GridSearchCV(rf_pipe, param_grid=param_grid, n_jobs=-1, scoring='f1_weighted', cv=inner_cv, verbose=10)

rf_nested_cv_f1_scores = cross_val_score(gs_cv, X, y, cv=outer_cv, verbose=10)

joblib.dump(rf_nested_cv_f1_scores, 'rf_nested_cv_f1_scores.pkl', compress=1)

print("##### Generalization F1 Score: mean =", np.mean(rf_nested_cv_f1_scores), "std =", np.std(rf_nested_cv_f1_scores))
print()

[CV]  ................................................................
Fitting 10 folds for each of 126 candidates, totalling 1260 fits
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200 .
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200 .
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200 .
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200, score=0.684493, total=   9.6s
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200, score=0.685318, total=   9.7s
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200, score=0.695087, total=   9.5s
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200 .
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimator

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   21.7s


[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200, score=0.688018, total=   9.5s
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200, score=0.687519, total=   9.6s
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200, score=0.679468, total=   9.3s
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350 .
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200, score=0.688009, total=   9.0s
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=200, score=0.691328, total=   9.0s
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350 .


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   32.4s


[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350, score=0.682378, total=  15.5s
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350, score=0.685077, total=  16.8s
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350, score=0.694181, total=  15.3s
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350, score=0.675445, total=  15.1s
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350, score=0.686634, total=  15.5s
[CV] rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=50, rf__n_estimators=350, score=0.687338, total=  16.8s
[CV] rf__max_depth=10, rf__min_samples_

...

[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=650, score=0.681391, total=  56.4s
[CV] rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=650 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=650, score=0.682513, total=  56.9s
[CV] rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=650 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=650, score=0.678887, total=  53.1s
[CV] rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=800 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=650, score=0.680706, total=  55.9s
[CV] rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=800 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=650, score=0.685589, total=  50.8s
[CV] rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=800 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=650, score=0.685681, total=  44.8s
[CV] rf__max_depth=10, rf__min_samples_

[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 35.7min


[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=950, score=0.689254, total=  59.5s
[CV] rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=950 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=950, score=0.674266, total=  59.8s
[CV] rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=950 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=950, score=0.682592, total=  56.9s
[CV] rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=950 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=950, score=0.683097, total=  56.7s
[CV] rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=950 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=950, score=0.679341, total=  54.6s
[CV] rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=1100 
[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=950, score=0.680276, total=  57.7s
[CV] rf__max_depth=10, rf__min_samples_

[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed: 41.1min


[CV]  rf__max_depth=10, rf__min_samples_leaf=80, rf__n_estimators=200, score=0.686268, total=  10.6s
[CV] rf__max_depth=10, rf__min_samples_leaf=80, rf__n_estimators=200 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=70, rf__n_estimators=1100, score=0.685309, total= 1.1min
[CV] rf__max_depth=10, rf__min_samples_leaf=80, rf__n_estimators=200 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=80, rf__n_estimators=200, score=0.683613, total=  12.2s
[CV] rf__max_depth=10, rf__min_samples_leaf=80, rf__n_estimators=200 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=80, rf__n_estimators=200, score=0.678865, total=  12.1s
[CV] rf__max_depth=10, rf__min_samples_leaf=80, rf__n_estimators=350 .
[CV]  rf__max_depth=10, rf__min_samples_leaf=80, rf__n_estimators=200, score=0.677343, total=  11.6s
[CV] rf__max_depth=10, rf__min_samples_leaf=80, rf__n_estimators=350 .


In [ ]:
# Function to plot boxplot of nested cross-validation scores.
def plot_cv_scores(sv_scores):
    sns.set_style("whitegrid")
    
    ax = sns.boxplot(x=cv_scores, orient="v")
    
    plt.show()

In [ ]:
lr_nested_cv_f1_scores = joblib.load()'lr_nested_cv_f1_scores.pkl')

plot_cv_scores(lr_nested_cv_f1_scores)

In [ ]:
def plot_variable_ranking(model, model_type, column_names):
    if model_type == "rf":
        importance = model.feature_importances_
        importance = pd.DataFrame(importance, index=column_names, columns=["Importance"])
        importance["Std"] = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
        
        x = range(importance.shape[0])
        y = importance.ix[:, 0]        
        yerr = importance.ix[:, 1]
        
        plt.bar(x, y, yerr=yerr, align="center")
        plt.xticks(range(len(column_names)), column_names)
        plt.show()
    elif model_type == "lr":
        importance = model.coef_
        importance = pd.DataFrame(importance, index=column_names, columns=["Importance"])
        
        x = range(importance.shape[0])
        y = importance.ix[:, 0]        
        
        plt.bar(x, y, align="center")
        plt.xticks(range(len(column_names)), column_names)
        plt.show()

In [ ]:
pca_lr_pipe = Pipeline([('variance', VarianceThreshold()),
                    ('scaler', StandardScaler()),
                    ('pca', PCA(random_state=554197)),
                    ('lr', LogisticRegression(class_weight='balanced', random_state=621473))])

param_grid = dict()
param_grid['lr__C'] = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid['pca__n_components'] = list(range(2, 14, 2))

inner_cv = StratifiedKFold(n_splits=10, random_state=975428)
outer_cv = StratifiedKFold(n_splits=10, random_state=248733)

gs_cv = GridSearchCV(pca_lr_pipe, param_grid=param_grid, n_jobs=-1, scoring='f1_weighted', cv=inner_cv, verbose=10)

pca_lr_nested_cv_f1_scores = cross_val_score(gs_cv, X, y, cv=outer_cv, verbose=10)

joblib.dump(pca_lr_nested_cv_f1_scores, 'pca_lr_nested_cv_f1_scores.pkl', compress=1)

print("##### Generalization F1 Score: mean =", np.mean(pca_lr_nested_cv_f1_scores), "std =", np.std(pca_lr_nested_cv_f1_scores))
print()

In [ ]:
rfe_lr_pipe = Pipeline([('variance', VarianceThreshold()),
                    ('scaler', StandardScaler()),
                    ('rfe', RFE(LogisticRegression(class_weight='balanced', random_state=348744), step=1)),
                    ('lr', LogisticRegression(class_weight='balanced', random_state=621473))])

param_grid = dict()
param_grid['lr__C'] = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid['rfe__n_features_to_select'] = list(range(5, 18, 5))
param_grid['rfe__estimator__C'] = [0.1, 1, 10]

inner_cv = StratifiedKFold(n_splits=10, random_state=975428)
outer_cv = StratifiedKFold(n_splits=10, random_state=248733)

gs_cv = GridSearchCV(rfe_lr_pipe, param_grid=param_grid, n_jobs=-1, scoring='f1_weighted', cv=inner_cv, verbose=10)

rfe_lr_nested_cv_f1_scores = cross_val_score(gs_cv, X, y, cv=outer_cv, verbose=10)

joblib.dump(rfe_lr_nested_cv_f1_scores, 'rfe_lr_nested_cv_f1_scores.pkl', compress=1)

print("##### Generalization F1 Score: mean =", np.mean(rfe_lr_nested_cv_f1_scores), "std =", np.std(rfe_lr_nested_cv_f1_scores))
print()